In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
import matplotlib.colors

In [ ]:
file_path_nutrition = "Nutrient_Density_Scores_Normalized_SubIndices.xlsx"
df_nutrition = pd.read_excel(file_path_nutrition)

category_avg_nds = df_nutrition.groupby('Category')['Normalized NDS'].mean().reset_index()

categorized_restaurants_path = 'Categorized_restaurants.xlsx'
nutrition_path = '外食計算.xlsx'
file_path_place = 'tokyo_unique_found_places.xlsx'

df_place = pd.read_excel(file_path_place)
df_restaurants = pd.read_excel(categorized_restaurants_path)
df_carbon = pd.read_excel(nutrition_path, sheet_name='carbon')

df_restaurants['FIES_category'] = df_restaurants['FIES_category'].astype(str)

df_restaurants = pd.merge(
    df_restaurants,
    df_nutrition,   
    left_on='FIES_category',
    right_on='Category',
    how='left'
)

df_restaurants = pd.merge(
    df_restaurants,
    df_place[['id', 'priceLevel', 'rating', 'userRatingCount', 'currentOpeningHours']],
    on='id',
    how='left'
)

df_merged = df_restaurants.merge(category_avg_nds, left_on='FIES_category', right_on='Category', how='left')

In [ ]:
file_path = 'Restaurant_500m_district.csv'
df = pd.read_csv(file_path)

df_merged = pd.merge(
    df,
    df_restaurants[['id', 'Normalized NDS', 'rating','priceLevel']], 
    how='left',        
    on='id'            
)

columns_to_drop = [
    'Energy_kca','Protein_g_','Lipid_g__x','Carbohydra','Salt_equiv',
    'Protein_g1','Lipid_g__y','Energy_k_1','Carbohyd_1','Salt_equ_1','Protein__1',
    'Lipid_g__1','Carbohyd_2','Salt_equ_2','NDS_Protei','NDS_Lipid',
    'NDS_Carboh','NDS_Salt_e','NDS','NDS_normal'
]

df_merged.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [ ]:
df_filtered = df_merged.drop_duplicates(subset=['N02_005', 'id'], keep='first')
df_filtered = df_filtered[df_filtered['rating'] > 0]

price_level_scores = {
    'PRICE_LEVEL_INEXPENSIVE': 0.7,
    'PRICE_LEVEL_MODERATE': 0.5,
    'PRICE_LEVEL_EXPENSIVE': 0.3,
    'PRICE_LEVEL_VERY_EXPENSIVE': 0.1
}
df_filtered['priceLevelScore'] = df_filtered['priceLevel'].map(price_level_scores)

average_price_scores = df_filtered.groupby('N02_005')['priceLevelScore'].mean()
average_rating_scores = df_filtered.groupby('N02_005')['rating'].mean()
df=df_filtered

restaurant_counts = df.groupby(['N02_005', 'primaryT_1']).size().unstack(fill_value=0)
restaurant_totals = restaurant_counts.sum(axis=1)

In [ ]:
proportions = restaurant_counts.div(restaurant_totals, axis=0)
shannon_indices = - (proportions * np.log(proportions)).sum(axis=1)

shannon_indices_normalized = shannon_indices / shannon_indices.max()
restaurant_totals_normalized = restaurant_totals / restaurant_totals.max()

composite_diversity_indices = restaurant_totals_normalized * shannon_indices_normalized
composite_diversity_indices2 = shannon_indices * restaurant_totals/restaurant_totals.max()/shannon_indices.max()

average_nutrition_scores = df.groupby('N02_005')['Normalized NDS'].mean()

composite_diversity_indices_root5 = np.power(composite_diversity_indices, 1/5)

In [ ]:
df_carbon_melted = df_carbon.melt(id_vars=['Type'], var_name='priceLevel', value_name='Carbon')

df_carbon_melted = df_carbon_melted.dropna(subset=['Carbon'])

merged_with_carbon = pd.merge(df, df_carbon_melted, left_on=['FIES_categ', 'priceLevel'], right_on=['Type', 'priceLevel'], how='left')

merged_with_carbon['Carbon'] = merged_with_carbon['Carbon'].fillna(0)

max_carbon = merged_with_carbon['Carbon'].max()
min_carbon = merged_with_carbon['Carbon'].min()

merged_with_carbon['Carbon_Normalized'] = (max_carbon - merged_with_carbon['Carbon']) / (max_carbon - min_carbon)

average_carbon_scores = merged_with_carbon.groupby('N02_005')['Carbon'].mean()
average_normalized_carbon_scores = merged_with_carbon.groupby('N02_005')['Carbon_Normalized'].mean()

average_normalized_carbon_scores4 = np.power(average_normalized_carbon_scores, 5)

In [ ]:
df_stations = pd.DataFrame({
    'Station': restaurant_totals.index,
    'Shannon_Index': shannon_indices,
    'Shannon_Index_Normalized': shannon_indices_normalized,
    'Restaurant_Total': restaurant_totals,
    'Restaurant_Total_Normalized': restaurant_totals_normalized,
    'Composite_Diversity_Index': composite_diversity_indices,
    'Average_Nutrition_Score': average_nutrition_scores,
    'Average_Price_Score': average_price_scores,
    'Average_Rating_Score': average_rating_scores,
    'Average_Carbon_Score': average_carbon_scores,
    'Average_Carbon_Score_Normalized': average_normalized_carbon_scores,
})

df_stations['Composite_Diversity_Index_Root5'] = composite_diversity_indices_root5.values

df_stations = df_stations.join(average_rating_scores, on='Station', rsuffix='_rating')

df_stations.rename(columns={'priceLevelScore': 'Average_Price_Score'}, inplace=True)

df_stations['Average_Carbon_Score_Normalized'] = df_stations['Station'].map(average_normalized_carbon_scores)

df_stations['Average_Carbon_Score_Normalized'].fillna(0, inplace=True)

station_info = df[['N02_005', 'NAME', 'latitude', 'longitude']].drop_duplicates('N02_005')

df_stations = pd.merge(df_stations, station_info, how='left', left_on='Station', right_on='N02_005')

In [ ]:
file_path = '500buffer_Table_population.xls'
df = pd.read_excel(file_path)

aggregated_data = df.groupby('S1222_NumberOfPassengers_Cli2.S12_001')['S1222_NumberOfPassengers_Cli2.S12_049'].sum().reset_index()

output_file_path = 'Population_processed.xlsx'
aggregated_data.to_excel(output_file_path, index=False)

In [ ]:
merged_data = df_stations.merge(aggregated_data, left_on='N02_005', right_on='S1222_NumberOfPassengers_Cli2.S12_001', how='left')
merged_data = merged_data[merged_data['S1222_NumberOfPassengers_Cli2.S12_049'] != 0]

merged_data['Crowdness'] = merged_data['Restaurant_Total'] / merged_data['S1222_NumberOfPassengers_Cli2.S12_049']

In [ ]:
from itertools import permutations
from math import sin, cos, pi

def calculate_area(values):
    # Convert the values into coordinates based on angles in a pentagon
    angles = np.linspace(0, 2 * np.pi, 5, endpoint=False)
    coordinates = [(v * cos(a), v * sin(a)) for v, a in zip(values, angles)]

    # Calculate area using the shoelace formula
    def shoelace(coords):
        x, y = zip(*coords)
        return 0.5 * abs(sum(x[i] * y[i + 1] - x[i + 1] * y[i] for i in range(-1, len(coords) - 1)))

    return shoelace(coordinates)

# Extract the required columns from merged_data
required_columns = ['Average_Nutrition_Score', 'Average_Price_Score', 'Average_Carbon_Score_Normalized4', 'Composite_Diversity_Index_Root5', 'Crowdness_Scaled']
data_for_area_calculation = merged_data[required_columns]

# Create a new DataFrame to store the results
results = []

# Calculate the area for each permutation for each row
for index, row in data_for_area_calculation.iterrows():
    row_permutations = list(set(permutations(row)))
    areas = [calculate_area(perm) for perm in row_permutations]
    average_area = np.mean(areas)
    merged_data.at[index, 'Average_Area'] = average_area
    for perm, area in zip(row_permutations, areas):
        results.append({'Index': index, 'Permutation': perm, 'Area': area})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)


In [ ]:
merged_df = pd.merge(df, merged_data, left_on='S1222_NumberOfPassengers_Cli2.S12_001', right_on='Station', how='left')

merged_df.to_excel('combined_data.xlsx', index=False)